In [101]:
import re
import subprocess

from astropy.io import fits
import pyfits

In [100]:
ls


29-06-2014UT-ARCSAT/  dataSet.py  LICENSE         masterlog.log
Basic-Notebook.ipynb  images.py   log_handler.py  README.md


In [146]:
def read_obs_log(logName):
    # The Latin-1 encoding is needed because of the degree symobl.
    log = open(logName,'r', encoding='Latin-1')
    log = log.read()
    return log

def extract_log_filenames(log):
    filenames = []
    searches = re.findall(r'(Imaging to) (\S+)', log)
    [filenames.append(x[1]) for x in searches[:]]
    return filenames

def compare_log_to_files():
    subprocess.call(['ls','-l'],
                   stdout=subprocess.STDOUT,universal_newlines=True)

In [147]:
log = read_arcsat_obs_log('masterlog.log')
filenames = extract_log_filenames(log)
compare_log_to_files()

OSError: [Errno 9] Bad file descriptor